# Data Exploration: Understanding the Tables

Before running experiments, we need to understand:
1. Table schemas
2. Data relationships
3. How to find **single-item packages** (where AI estimate matches actual measurement)

**Key Issue:**
- AI estimation is per item
- Actual weight/volume is per **combined package**
- We need orders where 1 item = 1 package

**Data Source:**
- **KSE** (KokuSai Express): `v2_kse_cost` - has actual measurements
- ~~FB (FastBox): `v2_fb_cost`~~ - no measurement data, cannot use

## 1. Setup

In [ ]:
!pip install -q google-cloud-bigquery pandas

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "sazoshop"
client = bigquery.Client(project=PROJECT_ID)

def run_query(sql):
    """Helper to run query and return DataFrame"""
    return client.query(sql).to_dataframe()

print("✅ Ready")

## 2. Explore Table Schemas

In [ ]:
# v2_order_items schema
schema_order_items = run_query("""
SELECT column_name, data_type, is_nullable
FROM `sazoshop.firestore_snapshot.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'v2_order_items'
ORDER BY ordinal_position
""")

print("=== v2_order_items schema ===")
print(f"Total columns: {len(schema_order_items)}")
schema_order_items

In [ ]:
# v2_kse_cost schema (KokuSai Express) - our measurement data source
schema_kse = run_query("""
SELECT column_name, data_type, is_nullable
FROM `sazoshop.firestore_collection.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'v2_kse_cost'
ORDER BY ordinal_position
""")

print("=== v2_kse_cost schema (KokuSai Express) ===")
print(f"Total columns: {len(schema_kse)}")
print("")
print("Key columns for experiment:")
print("  - order_id: join key")
print("  - dimensions: actual WxLxH (string)")
print("  - actual_weight: measured weight (kg)")
print("  - volumetric_weight: volumetric weight (kg)")
print("")
schema_kse

## 3. Sample Data Inspection

In [ ]:
# Sample from v2_order_items
sample_order_items = run_query("""
SELECT *
FROM `sazoshop.firestore_snapshot.v2_order_items`
LIMIT 3
""")

print("=== v2_order_items sample ===")
sample_order_items.T  # Transpose for readability

In [ ]:
# Sample from v2_kse_cost (with valid measurements)
sample_kse = run_query("""
SELECT *
FROM `sazoshop.firestore_collection.v2_kse_cost`
WHERE dimensions IS NOT NULL
  AND actual_weight IS NOT NULL
LIMIT 3
""")

print("=== v2_kse_cost sample (with measurements) ===")
sample_kse.T

In [ ]:
# Check dimensions format
print("=== Dimensions Format Examples ===")
for idx, row in sample_kse.iterrows():
    print(f"  {row['dimensions']} (weight: {row['actual_weight']} kg)")

## 4. Items per Order Distribution

In [ ]:
# How many items per order?
items_per_order = run_query("""
SELECT 
  item_count,
  COUNT(*) as order_count
FROM (
  SELECT 
    order_item_order_id,
    COUNT(DISTINCT order_item_id) as item_count
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
)
GROUP BY item_count
ORDER BY item_count
""")

print("=== Items per Order Distribution ===")
items_per_order

In [ ]:
# Visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.bar(items_per_order['item_count'].astype(str), items_per_order['order_count'])
plt.xlabel('Items per Order')
plt.ylabel('Number of Orders')
plt.title('Distribution: How many items per order?')
plt.show()

single_item_orders = items_per_order[items_per_order['item_count'] == 1]['order_count'].values[0]
total_orders = items_per_order['order_count'].sum()
print(f"\nSingle-item orders: {single_item_orders:,} / {total_orders:,} ({single_item_orders/total_orders*100:.1f}%)")

## 5. KSE Data Availability

In [ ]:
# Count KSE records
kse_counts = run_query("""
SELECT 
  COUNT(*) as total_records,
  COUNTIF(dimensions IS NOT NULL) as with_dimensions,
  COUNTIF(actual_weight IS NOT NULL) as with_actual_weight,
  COUNTIF(dimensions IS NOT NULL AND actual_weight IS NOT NULL) as with_both,
  COUNTIF(
    dimensions IS NOT NULL 
    AND actual_weight IS NOT NULL
    AND REGEXP_CONTAINS(dimensions, r'^[0-9.]+x[0-9.]+x[0-9.]+$')
  ) as with_valid_format
FROM `sazoshop.firestore_collection.v2_kse_cost`
""")

print("=== KSE Data Availability ===")
kse_counts.T

## 6. Single-Item Orders with KSE Measurements

In [ ]:
# Count: Single-item orders that have KSE shipping data
single_with_kse = run_query("""
WITH single_item_orders AS (
  SELECT order_item_order_id
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
  HAVING COUNT(DISTINCT order_item_id) = 1
)
SELECT 
  COUNT(*) as total_single_item_orders,
  COUNTIF(kse.order_id IS NOT NULL) as with_kse_data,
  COUNTIF(kse.dimensions IS NOT NULL AND kse.actual_weight IS NOT NULL) as with_measurements,
  COUNTIF(
    kse.dimensions IS NOT NULL 
    AND kse.actual_weight IS NOT NULL
    AND kse.actual_weight > 0
    AND REGEXP_CONTAINS(kse.dimensions, r'^[0-9.]+x[0-9.]+x[0-9.]+$')
  ) as with_valid_measurements
FROM single_item_orders sio
LEFT JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
  ON sio.order_item_order_id = kse.order_id
""")

print("=== Single-Item Orders with KSE Data ===")
single_with_kse.T

In [ ]:
# Sample valid single-item orders (our target dataset!)
valid_samples = run_query("""
WITH single_item_orders AS (
  SELECT 
    order_item_order_id,
    MAX(order_item_title_origin) as title,
    MAX(order_item_product_version_info_category) as category,
    MAX(ARRAY_TO_STRING(order_item_product_version_thumbnail_urls, '|')) as thumbnails,
    MAX(SAFE_CAST(JSON_EXTRACT_SCALAR(order_item_product_version_extra, '$.weight') AS FLOAT64)) as ai_weight,
    MAX(SAFE_CAST(JSON_EXTRACT_SCALAR(order_item_product_version_extra, '$.volume') AS FLOAT64)) as ai_volume
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
  HAVING COUNT(DISTINCT order_item_id) = 1
)
SELECT 
  sio.order_item_order_id,
  sio.title,
  sio.category,
  sio.ai_weight,
  sio.ai_volume,
  kse.actual_weight,
  kse.dimensions,
  kse.volumetric_weight
FROM single_item_orders sio
INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
  ON sio.order_item_order_id = kse.order_id
WHERE 
  kse.dimensions IS NOT NULL
  AND kse.actual_weight IS NOT NULL
  AND kse.actual_weight > 0
  AND REGEXP_CONTAINS(kse.dimensions, r'^[0-9.]+x[0-9.]+x[0-9.]+$')
LIMIT 20
""")

print("=== Valid Single-Item Orders (Target Dataset) ===")
print(f"Sample size: {len(valid_samples)}")
valid_samples

## 7. Data Quality Check

In [ ]:
# Check for outliers and data quality
data_quality = run_query("""
WITH single_item_orders AS (
  SELECT order_item_order_id
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
  HAVING COUNT(DISTINCT order_item_id) = 1
),
valid_data AS (
  SELECT 
    kse.actual_weight,
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(0)] AS FLOAT64) as width,
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(1)] AS FLOAT64) as length,
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(2)] AS FLOAT64) as height
  FROM single_item_orders sio
  INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
    ON sio.order_item_order_id = kse.order_id
  WHERE 
    kse.dimensions IS NOT NULL
    AND kse.actual_weight IS NOT NULL
    AND REGEXP_CONTAINS(kse.dimensions, r'^[0-9.]+x[0-9.]+x[0-9.]+$')
)
SELECT
  'actual_weight_kg' as metric,
  MIN(actual_weight) as min_val,
  ROUND(AVG(actual_weight), 2) as avg_val,
  MAX(actual_weight) as max_val,
  ROUND(APPROX_QUANTILES(actual_weight, 4)[OFFSET(2)], 2) as median_val
FROM valid_data
UNION ALL
SELECT
  'width_cm' as metric,
  MIN(width), ROUND(AVG(width), 1), MAX(width),
  ROUND(APPROX_QUANTILES(width, 4)[OFFSET(2)], 1)
FROM valid_data
UNION ALL
SELECT
  'length_cm' as metric,
  MIN(length), ROUND(AVG(length), 1), MAX(length),
  ROUND(APPROX_QUANTILES(length, 4)[OFFSET(2)], 1)
FROM valid_data
UNION ALL
SELECT
  'height_cm' as metric,
  MIN(height), ROUND(AVG(height), 1), MAX(height),
  ROUND(APPROX_QUANTILES(height, 4)[OFFSET(2)], 1)
FROM valid_data
""")

print("=== Data Quality: Value Ranges ===")
data_quality

In [ ]:
# Distribution of weights
weight_dist = run_query("""
WITH single_item_orders AS (
  SELECT order_item_order_id
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
  HAVING COUNT(DISTINCT order_item_id) = 1
)
SELECT 
  CASE 
    WHEN kse.actual_weight < 0.5 THEN '0-0.5kg'
    WHEN kse.actual_weight < 1 THEN '0.5-1kg'
    WHEN kse.actual_weight < 2 THEN '1-2kg'
    WHEN kse.actual_weight < 5 THEN '2-5kg'
    WHEN kse.actual_weight < 10 THEN '5-10kg'
    ELSE '10kg+'
  END as weight_range,
  COUNT(*) as count
FROM single_item_orders sio
INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
  ON sio.order_item_order_id = kse.order_id
WHERE kse.actual_weight IS NOT NULL AND kse.actual_weight > 0
GROUP BY weight_range
ORDER BY 
  CASE weight_range
    WHEN '0-0.5kg' THEN 1
    WHEN '0.5-1kg' THEN 2
    WHEN '1-2kg' THEN 3
    WHEN '2-5kg' THEN 4
    WHEN '5-10kg' THEN 5
    ELSE 6
  END
""")

print("=== Weight Distribution ===")
weight_dist

In [ ]:
# Visualize weight distribution
plt.figure(figsize=(10, 4))
plt.bar(weight_dist['weight_range'], weight_dist['count'])
plt.xlabel('Weight Range')
plt.ylabel('Count')
plt.title('Weight Distribution of Single-Item Orders')
plt.show()

## 8. Summary

In [ ]:
# Final summary
summary = run_query("""
WITH single_item_orders AS (
  SELECT order_item_order_id
  FROM `sazoshop.firestore_snapshot.v2_order_items`
  GROUP BY order_item_order_id
  HAVING COUNT(DISTINCT order_item_id) = 1
)
SELECT 
  COUNT(*) as valid_single_item_orders,
  COUNTIF(kse.actual_weight > 0 AND kse.actual_weight < 50) as reasonable_weight,
  COUNTIF(
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(0)] AS FLOAT64) < 200 AND
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(1)] AS FLOAT64) < 200 AND
    CAST(SPLIT(kse.dimensions, 'x')[OFFSET(2)] AS FLOAT64) < 200
  ) as reasonable_dimensions
FROM single_item_orders sio
INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
  ON sio.order_item_order_id = kse.order_id
WHERE 
  kse.dimensions IS NOT NULL
  AND kse.actual_weight IS NOT NULL
  AND REGEXP_CONTAINS(kse.dimensions, r'^[0-9.]+x[0-9.]+x[0-9.]+$')
""")

print("="*50)
print("SUMMARY: Valid Dataset for Experiments")
print("="*50)
print(f"")
print(f"Single-item orders with valid KSE data: {summary['valid_single_item_orders'].values[0]:,}")
print(f"With reasonable weight (0-50kg): {summary['reasonable_weight'].values[0]:,}")
print(f"With reasonable dimensions (<200cm): {summary['reasonable_dimensions'].values[0]:,}")
print(f"")
print("Data source: KSE (KokuSai Express) only")
print("(FB/FastBox lacks measurement data)")
print(f"")
print("These are suitable for AI estimation experiments!")

## Next Steps

Now that we understand the data:

1. **Use single-item orders only** - where actual measurement = item measurement
2. **Use KSE data only** - FB lacks measurement columns
3. **Filter outliers** - weight < 50kg, dimensions < 200cm
4. **Run experiments** with `weight_volume_experiment.ipynb`